In [1]:
import whisper
import time
import soundfile as sf
import numpy as np
import scipy

import torch
import torchaudio
from datasets import load_dataset
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor

print(torch.__version__)
print(torch.cuda.is_available())


c:\Users\User\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


2.9.1+cpu
False


In [2]:
cz_reference = "Modrý stín pomalu přeplouvá přes tichou hladinu jezera."

path_noise_snr0 = r"D:\prednasky\ARR\arr4\jezero_snr0.ogg"
path_noise_snr10 = r"D:\prednasky\ARR\arr4\jezero_snr10.ogg"
path_noise_snr20 = r"D:\prednasky\ARR\arr4\jezero_snr20.ogg"
path_noise_snr30 = r"D:\prednasky\ARR\arr4\jezero_snr30.ogg"


pavietrany_sar = r"D:\prednasky\ARR\arr4\pavietrany_šar.ogg"
jezero = r"D:\prednasky\ARR\arr4\jezero.ogg"

In [3]:
def add_noise(file, out_file, db):
    audio, sr = sf.read(file)
    snr_db = db
    signal_power = np.mean(audio**2)

    snr_linear = 10 ** (snr_db / 10)
    noise_power = signal_power / snr_linear

    noise = np.random.normal(0, np.sqrt(noise_power), audio.shape)

    noisy_audio = audio + noise

    sf.write(out_file, noisy_audio, sr)


def calculate_wer(reference, hypothesis):
	ref_words = reference.split()
	hyp_words = hypothesis.split()
	# Counting the number of substitutions, deletions, and insertions
	substitutions = sum(1 for ref, hyp in zip(ref_words, hyp_words) if ref != hyp)
	deletions = len(ref_words) - len(hyp_words)
	insertions = len(hyp_words) - len(ref_words)
	# Total number of words in the reference text
	total_words = len(ref_words)
	# Calculating the Word Error Rate (WER)
	wer = (substitutions + deletions + insertions) / total_words
	return wer


In [32]:
add_noise(jezero, path_noise_snr10, 10)
add_noise(jezero, path_noise_snr20, 20)
add_noise(jezero, path_noise_snr30, 30)
add_noise(jezero, path_noise_snr0, 0)

In [3]:
model = whisper.load_model("turbo")


In [33]:
curr1 = time.time()
result = model.transcribe(jezero)
curr2 = time.time()

wer = calculate_wer(cz_reference, result['text'])
noise_level = "-"

print("\n=== Výsledek rozpoznávání ===")
print(f"Rozpoznaný text : {result['text']}")
print(f"CZ reference    : {cz_reference}")

print("\n=== Statistiky ===")
print(f"{'Čas (s)':<15} {'WER':<10} {'SNR (dB)':<10}")
print(f"{curr2 - curr1:<15.3f} {wer:<10.3f} {noise_level:<10}")


curr1 = time.time()
result = model.transcribe(path_noise_snr0)
curr2 = time.time()

wer = calculate_wer(cz_reference, result['text'])
noise_level = "0"

print("\n=== Výsledek rozpoznávání ===")
print(f"Rozpoznaný text : {result['text']}")
print(f"CZ reference    : {cz_reference}")

print("\n=== Statistiky ===")
print(f"{'Čas (s)':<15} {'WER':<10} {'SNR (dB)':<10}")
print(f"{curr2 - curr1:<15.3f} {wer:<10.3f} {noise_level:<10}")



curr1 = time.time()
result = model.transcribe(path_noise_snr10)
curr2 = time.time()

wer = calculate_wer(cz_reference, result['text'])
noise_level = "10"

print("\n=== Výsledek rozpoznávání ===")
print(f"Rozpoznaný text : {result['text']}")
print(f"CZ reference    : {cz_reference}")

print("\n=== Statistiky ===")
print(f"{'Čas (s)':<15} {'WER':<10} {'SNR (dB)':<10}")
print(f"{curr2 - curr1:<15.3f} {wer:<10.3f} {noise_level:<10}")


curr1 = time.time()
result = model.transcribe(path_noise_snr20)
curr2 = time.time()

wer = calculate_wer(cz_reference, result['text'])
noise_level = "20"

print("\n=== Výsledek rozpoznávání ===")
print(f"Rozpoznaný text : {result['text']}")
print(f"CZ reference    : {cz_reference}")

print("\n=== Statistiky ===")
print(f"{'Čas (s)':<15} {'WER':<10} {'SNR (dB)':<10}")
print(f"{curr2 - curr1:<15.3f} {wer:<10.3f} {noise_level:<10}")



curr1 = time.time()
result = model.transcribe(path_noise_snr30)
curr2 = time.time()

wer = calculate_wer(cz_reference, result['text'])
noise_level = "30"

print("\n=== Výsledek rozpoznávání ===")
print(f"Rozpoznaný text : {result['text']}")
print(f"CZ reference    : {cz_reference}")

print("\n=== Statistiky ===")
print(f"{'Čas (s)':<15} {'WER':<10} {'SNR (dB)':<10}")
print(f"{curr2 - curr1:<15.3f} {wer:<10.3f} {noise_level:<10}")



=== Výsledek rozpoznávání ===
Rozpoznaný text :  Modrý stín pomalu přeplouvá přes tichou hladinu jezera.
CZ reference    : Modrý stín pomalu přeplouvá přes tichou hladinu jezera.

=== Statistiky ===
Čas (s)         WER        SNR (dB)  
16.951          0.000      -         

=== Výsledek rozpoznávání ===
Rozpoznaný text :  dass Nadiris v Istinn komaly vřeplhl doğ fully civil hladiny v šírov
CZ reference    : Modrý stín pomalu přeplouvá přes tichou hladinu jezera.

=== Statistiky ===
Čas (s)         WER        SNR (dB)  
63.134          1.000      0         

=== Výsledek rozpoznávání ===
Rozpoznaný text :  Modrý stín pomalu přeplouvá prostichou hladinu jezera.
CZ reference    : Modrý stín pomalu přeplouvá přes tichou hladinu jezera.

=== Statistiky ===
Čas (s)         WER        SNR (dB)  
16.469          0.375      10        

=== Výsledek rozpoznávání ===
Rozpoznaný text :  Modrý stín pomalu přeplouvá přes tichou hladinu jezera.
CZ reference    : Modrý stín pomalu přeplouvá přes tic

In [9]:
curr1 = time.time()
result = model.transcribe(pavietrany_sar)
curr2 = time.time()
print(result)

print("It takes:", curr2-curr1)

{'text': ' Глядеть на брутный город и кремзать мемуары, Не варта вачей и не варта рук, Я гляджу на зоры, я гляджу на хмары, И слухаю власного сердца стук.', 'segments': [{'id': 0, 'seek': 0, 'start': 1.0, 'end': 4.12, 'text': ' Глядеть на брутный город и кремзать мемуары,', 'tokens': [50415, 7247, 2873, 856, 9108, 1470, 1268, 47220, 4441, 18750, 1006, 981, 7049, 1544, 2209, 1084, 8339, 48632, 11, 50571], 'temperature': 0.0, 'avg_logprob': -0.22980465760102142, 'compression_ratio': 1.624203821656051, 'no_speech_prob': 1.418735848801589e-07}, {'id': 1, 'seek': 0, 'start': 4.58, 'end': 6.62, 'text': ' Не варта вачей и не варта рук,', 'tokens': [50594, 11512, 740, 2222, 10788, 740, 8314, 2345, 1006, 1725, 740, 2222, 10788, 36765, 11, 50696], 'temperature': 0.0, 'avg_logprob': -0.22980465760102142, 'compression_ratio': 1.624203821656051, 'no_speech_prob': 1.418735848801589e-07}, {'id': 2, 'seek': 0, 'start': 6.86, 'end': 9.76, 'text': ' Я гляджу на зоры, я гляджу на хмары,', 'tokens': [5070

In [4]:
processor = Wav2Vec2Processor.from_pretrained("arampacha/wav2vec2-large-xlsr-czech",
                                               cache_dir=r"D:\prednasky\ARR\arr4")
model = Wav2Vec2ForCTC.from_pretrained("arampacha/wav2vec2-large-xlsr-czech",
                                               cache_dir=r"D:\prednasky\ARR\arr4")


# 1. Загружаем OGG файл
speech_array, sampling_rate = torchaudio.load(jezero)
speech_array = speech_array.squeeze()

print("Оригинальная частота:", sampling_rate)

# 2. Если частота ≠ 16000 → ресемплим
if sampling_rate != 16000:
    resampler = torchaudio.transforms.Resample(sampling_rate, 16000)
    speech_array = resampler(speech_array)
    sampling_rate = 16000

# 3. Конвертируем в numpy
speech = speech_array.numpy()

# 4. Передаём в модель
inputs = processor(speech, sampling_rate=16000, return_tensors="pt", padding=True)

with torch.no_grad():
    logits = model(inputs.input_values).logits

pred_ids = torch.argmax(logits, dim=-1)
text = processor.decode(pred_ids[0])

print("Распознанный текст:", text)

c:\Users\User\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\configuration_utils.py:335: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


ImportError: TorchCodec is required for load_with_torchcodec. Please install torchcodec to use this function.